In [49]:
import requests
import plotly.express as pe
import pandas as pd

In [50]:

# host = 'http://localhost/greenatlas'
host = 'https://www.mygreenatlas.com'
lifestyle_posts = requests.get(f'{host}/wp-json/wp/v2/lifestyles', timeout=600)
products_posts = requests.get(f'{host}/wp-json/wp/v2/products', timeout=600)


## Process the Data

In [51]:
impact_data = []

posts = []
posts.extend(lifestyle_posts.json())
posts.extend(products_posts.json())
for lifestyle in posts:
    id = lifestyle['id']

    impact_resp = requests.get(f'{host}/wp-json/greenatlas/v1/calculator/{id}')

    name = lifestyle['title']['rendered']
    link = lifestyle['link']
    
    impact = impact_resp.json()
    if impact['IsDefined']:
        impact_data.append({
            "label": f"<a href={link}>{name}</a>",
            "name": name,
            "link": link,
            **impact
        })

print(impact_data)

[{'label': '<a href=https://www.mygreenatlas.com/lifestyles/sustainablebirthdayparties/>Sustainable Birthday Parties</a>', 'name': 'Sustainable Birthday Parties', 'link': 'https://www.mygreenatlas.com/lifestyles/sustainablebirthdayparties/', 'DollarsPerYearBenefit': {'Min': 0, 'Max': 0}, 'DollarsUpfrontBenefit': {'Min': 0, 'Max': 0}, 'TonsCO2ePerYearBenefit': {'Min': 0.4564979999999999, 'Max': 0.8477819999999999}, 'TonsCO2eUpfrontBenefit': {'Min': 0, 'Max': 0}, 'HoursPerYearBenefit': {'Min': -6, 'Max': -18}, 'HoursUpfrontBenefit': {'Min': -2, 'Max': -8}, 'IsDefined': True}, {'label': '<a href=https://www.mygreenatlas.com/lifestyles/sustainablecoffee/>Sustainable Coffee</a>', 'name': 'Sustainable Coffee', 'link': 'https://www.mygreenatlas.com/lifestyles/sustainablecoffee/', 'DollarsPerYearBenefit': {'Min': 0, 'Max': 0}, 'DollarsUpfrontBenefit': {'Min': -10, 'Max': -50}, 'TonsCO2ePerYearBenefit': {'Min': 0.0240646, 'Max': 0.0446914}, 'TonsCO2eUpfrontBenefit': {'Min': 0, 'Max': 0}, 'Hours

In [52]:
impact_df = pd.json_normalize(impact_data)

def average_df(DF: pd.DataFrame, name: str):
    DF[name] = DF[[f'{name}.Min',f'{name}.Max']].mean(axis=1)
    DF[f'{name}.Range'] = ( DF[f'{name}.Max'] - DF[f'{name}.Min'] ) / 2

average_these = [
    'TonsCO2ePerYearBenefit','TonsCO2eUpfrontBenefit',
    'DollarsPerYearBenefit','DollarsUpfrontBenefit',
    'HoursPerYearBenefit','HoursUpfrontBenefit'
]

for average_me in average_these:
    average_df(impact_df, average_me)


## Plots

In [53]:

plt1 = pe.scatter(data_frame=impact_df, x='DollarsPerYearBenefit', y='TonsCO2ePerYearBenefit', color='HoursPerYearBenefit', size='TonsCO2ePerYearBenefit.Range', text='name')
plt1.show()

plt2 = pe.scatter(data_frame=impact_df, x='HoursPerYearBenefit', y='TonsCO2ePerYearBenefit', color='DollarsPerYearBenefit',size='TonsCO2ePerYearBenefit.Range')
plt2.show()

In [58]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=impact_df["DollarsPerYearBenefit"],
        y=impact_df["TonsCO2ePerYearBenefit"],
        marker=dict(
            size=impact_df["TonsCO2ePerYearBenefit.Range"]**2 * 3.1415926, color=impact_df['HoursPerYearBenefit']
        ),
        text=impact_df["label"],
        mode="markers+text",
        name="Blah",
        textposition="top center",
    )
)
# pi*r^2 = size